# Dynamodb

## Create

In [5]:
import boto3

# Get the service resource.
dynamodb = boto3.resource('dynamodb')

# Create the DynamoDB table.
table = dynamodb.create_table(
    TableName='users',
    KeySchema=[
        {
            'AttributeName': 'username',
            'KeyType': 'HASH'
        },
        {
            'AttributeName': 'last_name',
            'KeyType': 'RANGE'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'username',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'last_name',
            'AttributeType': 'S'
        },
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

# Wait until the table exists.
table.meta.client.get_waiter('table_exists').wait(TableName='users')

# Print out some data about the table.
print(table.item_count)

0


## Search, Query

In [6]:
import boto3

# Get the service resource.
dynamodb = boto3.resource('dynamodb')

# Instantiate a table resource object without actually
# creating a DynamoDB table. Note that the attributes of this table
# are lazy-loaded: a request is not made nor are the attribute
# values populated until the attributes
# on the table resource are accessed or its load() method is called.
table = dynamodb.Table('users')

# Print out some data about the table.
# This will cause a request to be made to DynamoDB and its attribute
# values will be set based on the response.
print(table.creation_date_time)

2021-11-20 10:56:37.743000+00:00


In [11]:
#Creating a new item
table.put_item(
   Item={
        'username': 'janedoe',
        'first_name': 'Jane',
        'last_name': 'Doe',
        'age': 25,
        'account_type': 'standard_user',
    }
)
# Getting an item
response = table.get_item(
    Key={
        'username': 'janedoe',
        'last_name': 'Doe'
    }
)
item = response['Item']
print(item)

# Updating an item
table.update_item(
    Key={
        'username': 'janedoe',
        'last_name': 'Doe'
    },
    UpdateExpression='SET age = :val1',
    ExpressionAttributeValues={
        ':val1': 26
    }
)
response = table.get_item(
    Key={
        'username': 'janedoe',
        'last_name': 'Doe'
    }
)
item = response['Item']
print(item)

# Deleting an item
table.delete_item(
    Key={
        'username': 'janedoe',
        'last_name': 'Doe'
    }
)

{'username': 'janedoe', 'account_type': 'standard_user', 'last_name': 'Doe', 'age': Decimal('25'), 'first_name': 'Jane'}
{'username': 'janedoe', 'account_type': 'standard_user', 'last_name': 'Doe', 'age': Decimal('26'), 'first_name': 'Jane'}


{'ResponseMetadata': {'RequestId': 'GE38T2RMO6D4UFLDUMB8E38I4VVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sat, 20 Nov 2021 11:01:47 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'GE38T2RMO6D4UFLDUMB8E38I4VVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [12]:
# Batch writing
with table.batch_writer() as batch:
    for i in range(50):
        batch.put_item(
            Item={
                'account_type': 'anonymous',
                'username': 'user' + str(i),
                'first_name': 'unknown',
                'last_name': 'unknown'
            }
        )

In [17]:
# Querying and scanning
from boto3.dynamodb.conditions import Key, Attr

table.put_item(
   Item={
        'username': 'janedoe',
        'first_name': 'Jane',
        'last_name': 'Doe',
        'age': 25,
        'account_type': 'standard_user',
    }
)
response = table.query(
    KeyConditionExpression=Key('username').eq('janedoe')
)
items = response['Items']
print(items)

[{'username': 'janedoe', 'account_type': 'standard_user', 'last_name': 'Doe', 'first_name': 'Jane', 'age': Decimal('25')}]


In [19]:
# lt表示less than小于27岁
response = table.scan(
    FilterExpression=Attr('age').lt(27)
)
items = response['Items']
print(items)

[{'username': 'janedoe', 'account_type': 'standard_user', 'last_name': 'Doe', 'first_name': 'Jane', 'age': Decimal('25')}]


In [21]:
response = table.scan(
    FilterExpression=Attr('first_name').begins_with('J')
)
items = response['Items']
print(items)

[{'username': 'janedoe', 'account_type': 'standard_user', 'last_name': 'Doe', 'first_name': 'Jane', 'age': Decimal('25')}]


In [22]:
response = table.scan(
    FilterExpression=Attr('first_name').eq('Jane')
)
items = response['Items']
print(items)

[{'username': 'janedoe', 'account_type': 'standard_user', 'last_name': 'Doe', 'first_name': 'Jane', 'age': Decimal('25')}]


In [23]:
table.delete()

{'TableDescription': {'TableName': 'users',
  'TableStatus': 'DELETING',
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 5,
   'WriteCapacityUnits': 5},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:eu-west-1:071908484098:table/users',
  'TableId': '40d66e66-e11d-493a-94a3-fd30609f0abc'},
 'ResponseMetadata': {'RequestId': 'DCFR8CR6F2BAF89LEG5RP5A8LVVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sat, 20 Nov 2021 11:05:53 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '312',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'DCFR8CR6F2BAF89LEG5RP5A8LVVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2433127517'},
  'RetryAttempts': 0}}

# Create an RDS instance
First, we need to create a Security Group allowing incoming connections to port 3306 (the default MySQL port).

https://boto3.readthedocs.io/en/latest/reference/services/ec2.html#EC2.Client.create_security_group https://boto3.readthedocs.io/en/latest/reference/services/ec2.html#EC2.Client.authorize_security_group_ingress

Then, we launch an RDS MySQL instance.

https://boto3.readthedocs.io/en/latest/reference/services/rds.html#RDS.Client.create_db_instance

Make sure to use the 'db.t2.micro' instance class to stay within the free tier.

In [7]:
import boto3

ec2 = boto3.client('ec2', region_name='eu-west-1')
response = ec2.create_security_group(GroupName='meetupdbsg',Description='SG for ML Meetup')
sgid = response['GroupId']
ec2.authorize_security_group_ingress(GroupId=sgid,IpProtocol='tcp',CidrIp='0.0.0.0/0',FromPort=3306,ToPort=3306)

rds_client = boto3.client('rds', region_name='eu-west-1')
rds_client.create_db_instance(
    DBName='meetupdb', DBInstanceIdentifier='meetuprdsinstance', 
    Engine='mysql', EngineVersion='8.0.23', DBInstanceClass='db.t2.micro', AllocatedStorage=8,
    MasterUsername='julien', MasterUserPassword='julienjulien',
    VpcSecurityGroupIds=[sgid])

{'DBInstance': {'DBInstanceIdentifier': 'meetuprdsinstance',
  'DBInstanceClass': 'db.t2.micro',
  'Engine': 'mysql',
  'DBInstanceStatus': 'creating',
  'MasterUsername': 'julien',
  'DBName': 'meetupdb',
  'AllocatedStorage': 8,
  'PreferredBackupWindow': '02:11-02:41',
  'BackupRetentionPeriod': 1,
  'DBSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0c35483c936d103ad',
    'Status': 'active'}],
  'DBParameterGroups': [{'DBParameterGroupName': 'default.mysql8.0',
    'ParameterApplyStatus': 'in-sync'}],
  'DBSubnetGroup': {'DBSubnetGroupName': 'default',
   'DBSubnetGroupDescription': 'default',
   'VpcId': 'vpc-00b23a07e7d9d1217',
   'SubnetGroupStatus': 'Complete',
   'Subnets': [{'SubnetIdentifier': 'subnet-0da174ec16a5daded',
     'SubnetAvailabilityZone': {'Name': 'eu-west-1a'},
     'SubnetOutpost': {},
     'SubnetStatus': 'Active'},
    {'SubnetIdentifier': 'subnet-084ae15ea4463ce20',
     'SubnetAvailabilityZone': {'Name': 'eu-west-1b'},
     'Subnet

In [15]:
import boto3

rds_client = boto3.client('rds', region_name='eu-west-1')
instances = rds_client.describe_db_instances()
for i in instances['DBInstances']:
    print ('Name: ' + i['DBInstanceIdentifier'])
    print ('Status: ' + i['DBInstanceStatus'])
    print ('Endpoint: ' + i['Endpoint']['Address'])
    print ('Port: ' + str(i['Endpoint']['Port']))

Name: meetuprdsinstance
Status: available
Endpoint: meetuprdsinstance.c68wfce0e2bb.eu-west-1.rds.amazonaws.com
Port: 3306


In [18]:
for i in instances['DBInstances']:
    print('Instance:', i["DBInstanceIdentifier"])
    print('DB Name:', i["DBName"])
    print('Status:', i["DBInstanceStatus"])
    print('Engine:', i['Engine'], i['EngineVersion'])
    try:
        print('Port:', i["Endpoint"]['Port'])
        print('Endpoint:', i["Endpoint"]['Address'])
        print('Security Group:', i["VpcSecurityGroups"][0]["VpcSecurityGroupId"], i["VpcSecurityGroups"][0]['Status'] )
    except:
        pass
        print('-' * 50)

Instance: meetuprdsinstance
DB Name: meetupdb
Status: available
Engine: mysql 8.0.23
Port: 3306
Endpoint: meetuprdsinstance.c68wfce0e2bb.eu-west-1.rds.amazonaws.com
Security Group: sg-0c35483c936d103ad active


In [17]:
# Get Snapshots
rds_client.describe_db_snapshots()

{'DBSnapshots': [{'DBSnapshotIdentifier': 'mythical-mysfits-devsecops-snapshot-clairdb-i7xj65g700c5',
   'DBInstanceIdentifier': 'mcq8lk90t6n8hb',
   'SnapshotCreateTime': datetime.datetime(2021, 11, 12, 0, 7, 23, 920000, tzinfo=tzlocal()),
   'Engine': 'postgres',
   'AllocatedStorage': 20,
   'Status': 'available',
   'Port': 5432,
   'AvailabilityZone': 'eu-west-1a',
   'VpcId': 'vpc-08cca75e48ee0c810',
   'InstanceCreateTime': datetime.datetime(2021, 11, 7, 22, 25, 51, 80000, tzinfo=tzlocal()),
   'MasterUsername': 'postgres',
   'EngineVersion': '9.6.22',
   'LicenseModel': 'postgresql-license',
   'SnapshotType': 'manual',
   'OptionGroupName': 'default:postgres-9-6',
   'PercentProgress': 100,
   'StorageType': 'gp2',
   'Encrypted': False,
   'DBSnapshotArn': 'arn:aws:rds:eu-west-1:071908484098:snapshot:mythical-mysfits-devsecops-snapshot-clairdb-i7xj65g700c5',
   'IAMDatabaseAuthenticationEnabled': False,
   'ProcessorFeatures': [],
   'DbiResourceId': 'db-FMWH7GAW7BU5R5YZCHIU

# Delete an RDS instance
When you're done working with your instance, don't forget to delete it. Be careful, there is no confirmation!

You can use the describe_db_instances() script above to check that your instance is being deleted.



In [ ]:
import boto3

rds_client = boto3.client('rds', region_name='eu-west-1')
response = rds_client.delete_db_instance(DBInstanceIdentifier='meetuprdsinstance',SkipFinalSnapshot=True)
print response